# [TUTORIAL](https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_Fundamental.ipynb)

Este notebook tem como objetivo treinar 5 modelos de reforce learning para os dados fundamentalistas das acoes do ibovespa

# 1.Importando as bibliotecas

In [1]:
#!pip list > pacotes_instalados_erro.txt

In [2]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
#!pip install pandas==1.5.3

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-qp6flhsi
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-qp6flhsi
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 453349afc2ee5eac8cc7f5766638c50f595df2d0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-_uy_eypm/elegantrl_e84b60efe04648d08ee929b8ab32d1ae
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-_uy_eypm/elegantrl_e84b60efe04648d08ee929b8ab32d1ae
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b4b9d662b9f9cb7cc368ac2b1036b5119eb20be4
  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import yfinance as yf

In [4]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

In [5]:
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import sys
sys.path.append("../Finrl_fundamentalist_trader")

In [7]:
import itertools

from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

**Criando pastas**

In [8]:
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [9]:
"""
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
#from finrl.apps import config
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools
"""

'\n!pip install wrds\n!pip install swig\n!pip install -q condacolab\nimport condacolab\ncondacolab.install()\n!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig\n!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git\n\nimport pandas as pd\nimport numpy as np\nimport matplotlib\nimport matplotlib.pyplot as plt\n# matplotlib.use(\'Agg\')\nimport datetime\n\n%matplotlib inline\n#from finrl.apps import config\nfrom finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader\nfrom finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split\nfrom finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv\nfrom finrl.drl_agents.stablebaselines3.models import DRLAgent\nfrom finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline\n\nfrom pprint import pprint\n\nimport sys\nsys.path.append("../FinRL-Library")\n\nimport itertools\n'

# 2.Importando os dados

Foi salvado assim [prices.to_hdf('dados_ibovespa.h5','dados_fun_tec')]

In [10]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
diretorio_computador = 'C:/Users/jefer/Documents/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/dados_fun_tec.csv'
diretorio_drive = '/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/dados_fun_tec.csv'

In [12]:
try:
    prices = (pd.read_csv(diretorio_computador)
              .reset_index()
              .rename(columns={'data': 'date'}))
except:
    prices = (pd.read_csv(diretorio_drive)
              .reset_index()
              .rename(columns={'data': 'date'}))

In [13]:
drop_colunas = ['r42','r21', 'month','r63dec','retorno_sobre_patrimonio_liquido_inicial','retorno_sobre_capital_investido_inicial_pre_impostos',
              'r05dec','r42dec','r21dec','r10dec','depreciacao_amortizacao','market_cap_empresa','ativo_total','acoes_ordinarias','divida_bruta','margem_ebitda','ativo_total', 'acoes_ordinarias','margem_bruta','patrimonio_liquido',
                'ev_atito_total','enterprise_value','ebitida', 'r01dec','ev_ebit','ev_receita_liquida','ev_fco','margem_liquida']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
prices.drop(drop_colunas,axis=1,inplace=True)

In [15]:
prices.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['index', 'tic', 'date', 'open', 'high', 'low', 'close', 'volume', 'day',
       'receita_liquida', 'resultado_bruto', 'lucro_liquido', 'lucro_por_acao',
       'preco_lucro', 'preco_vpa', 'preco_receita_liquida', 'preco_fco',
       'preco_ativo_total', 'preco_ebit', 'preco_capital_giro', 'preco_ncav',
       'dividend_yield', 'caixa_equivalentes_caixa', 'divida_curto_prazo',
       'divida_longo_razo', 'divida_liquida', 'valor_patrimonial_acao',
       'retorno_sobre_ativo_inicial', 'margem_ebit',
       'market_cap_empresa_categoria', 'pe_diario', 'preco_resultado_bruto',
       'preco_ebit_diario', 'preco_ativo_total_diario',
       'preco_divida_bruta_diaria', 'preco_divida_liquida_diaria',
       'preco_valor_patrimonial_diaria', 'porcentagem_acoes_ordinarias',
       'setor', 'rank_volume_medio', 'rsi', 'bb_high', 'bb_low', 'NATR', 'ATR',
       'PPO', 'MACD', 'r01', 'r05', 'r10', 'r63', 'weekday'],
      dtype='object')

In [16]:
prices = prices.iloc[:,1:]

In [17]:
prices

,tic,date,open,high,low,close,volume,day,receita_liquida,resultado_bruto,...,bb_low,NATR,ATR,PPO,MACD,r01,r05,r10,r63,weekday
0,ABEV3,2000-02-29,16.980267,17.050261,16.768290,11.399803,1126114.0,2.0,11960.0,7180.00,...,0.000000,49.566271,0.697767,0.000000,-0.013664,0.000000,0.000000,0.000000,0.000000,1
1,ABEV3,2000-03-01,16.980267,17.050261,16.768290,11.399803,1126114.0,2.0,11960.0,7180.00,...,0.000000,49.566271,0.697767,0.000000,-0.013664,0.000000,0.000000,0.000000,0.000000,2
2,ABEV3,2000-03-02,16.980267,17.050261,16.768290,11.399803,1126114.0,2.0,11960.0,7180.00,...,0.000000,49.566271,0.697767,0.000000,-0.013664,0.000000,0.000000,0.000000,0.000000,3
3,ABEV3,2000-03-03,16.980267,17.050261,16.768290,11.399803,1126114.0,2.0,11960.0,7180.00,...,0.000000,49.566271,0.697767,0.000000,-0.013664,0.000000,0.000000,0.000000,0.000000,4
4,ABEV3,2000-03-04,16.980267,17.050261,16.768290,11.399803,1126114.0,2.0,11960.0,7180.00,...,0.000000,49.566271,0.697767,0.000000,-0.013664,0.000000,0.000000,0.000000,0.000000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439309,YDUQ3,2023-09-25,19.250000,19.389999,18.820000,19.040001,3672200.0,4.0,1310.0,757.62,...,0.001452,5.210161,-0.509576,-1.876841,-0.941093,-0.013982,-0.056492,-0.059753,-0.090323,0
439310,YDUQ3,2023-09-26,18.889999,19.510000,18.860001,19.000000,5959900.0,1.0,1310.0,757.62,...,0.008992,5.092553,-0.522575,-2.401406,-1.008372,-0.002101,-0.032094,-0.061728,-0.081396,1
439311,YDUQ3,2023-09-27,19.150000,19.639999,18.719999,18.870001,7060400.0,2.0,1310.0,757.62,...,0.012227,5.109624,-0.524384,-2.554140,-1.070837,-0.006842,-0.022786,-0.068148,-0.081541,2
439312,YDUQ3,2023-09-28,19.030001,19.600000,18.600000,19.600000,6025700.0,3.0,1310.0,757.62,...,0.050104,4.932369,-0.523023,-2.542115,-0.990148,0.038686,0.015018,-0.075908,-0.034404,3


## Dividindo treino e teste

**Pegando vix e turbulencia necessarios para o ambiente**

In [18]:
fe = FeatureEngineer(use_technical_indicator=False,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
prices_preproces = fe.preprocess_data(prices)

[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (5934, 8)
Successfully added vix
Successfully added turbulence index


In [20]:
len(prices.tic.unique())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


51

**Porcentagem de treino**

In [21]:
porcentagem_treino = 0.8

**O numero o index limite**

In [22]:
index_data_inical = int(len(prices_preproces.date.unique())*porcentagem_treino)
index_data_inical

4747

**A data limite**

In [23]:
data_limite = prices_preproces.date.unique()[index_data_inical]
data_limite

'2019-01-11'

In [24]:
treino_inicio = '2010-01-11'
treino_final =  prices_preproces['date'].unique()[:index_data_inical].max()
teste_inicio = prices_preproces['date'].unique()[index_data_inical:].min()
teste_final = prices_preproces['date'].unique()[index_data_inical:].max()

In [25]:
treino = data_split(prices_preproces, treino_inicio,treino_final)
trade = data_split(prices_preproces, teste_inicio,teste_final)

In [26]:
print('Porcentagem de',porcentagem_treino, 'para o treinamento e', round(1-porcentagem_treino,1),'para o teste')
print('Total de',len(treino),'de dados para o treinamento e', len(trade),'para o teste')
print('Data do incio do treino:',treino.date.unique().min())
print('Data do termino do treino:',treino.date.unique().max())
print('Data do incio do teste:',trade.date.unique().min())
print('Data do termino do teste:',trade.date.unique().max())

Porcentagem de 0.8 para o treinamento e 0.2 para o teste
Total de 115515 de dados para o treinamento e 60486 para o teste
Data do incio do treino: 2010-01-11
Data do termino do treino: 2019-01-09
Data do incio do teste: 2019-01-11
Data do termino do teste: 2023-09-27


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Salvando os dados**

In [27]:
try:
    treino.to_csv('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/treino_fund.csv')
    trade.to_csv('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trade_fund.csv')
except:
    treino.to_csv('treino_fund.csv')
    trade.to_csv('trade_fund.csv')

In [28]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115515 entries, 0 to 2264
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   tic                             115515 non-null  object 
 1   date                            115515 non-null  object 
 2   open                            115515 non-null  float64
 3   high                            115515 non-null  float64
 4   low                             115515 non-null  float64
 5   close                           115515 non-null  float64
 6   volume                          115515 non-null  float64
 7   day                             115515 non-null  float64
 8   receita_liquida                 115515 non-null  float64
 9   resultado_bruto                 115515 non-null  float64
 10  lucro_liquido                   115515 non-null  float64
 11  lucro_por_acao                  115515 non-null  float64
 12  preco_lucro       

# 3.Criando o ambiente

O [ambiente](https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_Fundamental.ipynb) de negociação de ações criado para ser utilizado com a biblioteca OpenAI Gym


In [29]:
valor_inicial_carteira = 10000
custo_trade = [0.01]
taxa_aprendizagem = 0.01
iteracao = 50000

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
indicadores = treino.columns[8:]
indicadores

Index(['receita_liquida', 'resultado_bruto', 'lucro_liquido', 'lucro_por_acao',
       'preco_lucro', 'preco_vpa', 'preco_receita_liquida', 'preco_fco',
       'preco_ativo_total', 'preco_ebit', 'preco_capital_giro', 'preco_ncav',
       'dividend_yield', 'caixa_equivalentes_caixa', 'divida_curto_prazo',
       'divida_longo_razo', 'divida_liquida', 'valor_patrimonial_acao',
       'retorno_sobre_ativo_inicial', 'margem_ebit',
       'market_cap_empresa_categoria', 'pe_diario', 'preco_resultado_bruto',
       'preco_ebit_diario', 'preco_ativo_total_diario',
       'preco_divida_bruta_diaria', 'preco_divida_liquida_diaria',
       'preco_valor_patrimonial_diaria', 'porcentagem_acoes_ordinarias',
       'setor', 'rank_volume_medio', 'rsi', 'bb_high', 'bb_low', 'NATR', 'ATR',
       'PPO', 'MACD', 'r01', 'r05', 'r10', 'r63', 'weekday', 'vix',
       'turbulence'],
      dtype='object')

In [31]:
stock_dimension = len(treino.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicadores)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 51, State Space: 2398


In [32]:
buy_cost_list = sell_cost_list = custo_trade * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount":valor_inicial_carteira ,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicadores,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = treino, **env_kwargs)

In [33]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# 4.Treinando os modelos
Serao treinados 5 modelos da biblioteca **[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)**

Os modelos de aprendizagem de reforco são o: [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html), [DDPG](https://stable-baselines3.readthedocs.io/en/master/modules/ddpg.html), [PPO](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html), [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html).

## Configurações do ambiente

In [34]:
agent = DRLAgent(env = env_train)

# Colque true para o algoritimo que vc quer treinar
if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

## Treinando o algoritimo A2C

In [35]:
agent = DRLAgent(env = env_train)
A2C_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": taxa_aprendizagem
}
model_a2c = agent.get_model("a2c",model_kwargs=A2C_PARAMS)


if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.01}
Using cpu device


In [36]:
start_time = time.time()
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=iteracao) if if_using_a2c  else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
try:
    trained_a2c.save('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trained_models/agent_a2c_fundamentos') if if_using_a2c else None

except:
    trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c_fundamentos") if if_using_a2c  else None

## Treinando o algoritimo DDPG

In [38]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {
    "learning_rate": taxa_aprendizagem,
    "batch_size": 128,
}
model_ddpg = agent.get_model("ddpg",model_kwargs=DDPG_PARAMS)

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'learning_rate': 0.01, 'batch_size': 128}
Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 19.40GB > 11.39GB
  warnings.warn(


In [39]:
start_time = time.time()

trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=iteracao) if if_using_ddpg else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [40]:
try:
    trained_ddpg.save('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trained_models/agent_ddpg_fundamentos') if if_using_ddpg else None

except:
    trained_ddpg.save(TRAINED_MODEL_DIR + "/agent_ddpg_fundamentos") if if_using_ddpg else None

## Treinando o algoritimo PPO

In [41]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": taxa_aprendizagem,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.01, 'batch_size': 128}
Using cpu device


In [42]:
start_time = time.time()
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=iteracao) if if_using_ppo else None

end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [43]:
try:
    trained_ppo.save('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trained_models/agent_ppo_fundamentos') if if_using_ppo else None

except:
    trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo_fundamentos") if if_using_ppo else None

## Treinando o algoritimo TD3

In [44]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": taxa_aprendizagem}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 19.40GB > 11.37GB
  warnings.warn(


{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.01}
Using cpu device


In [45]:
start_time = time.time()
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=iteracao) if if_using_td3 else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [46]:
try:
   trained_td3.save('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trained_models/agent_td3_fundamentos') if if_using_td3 else None

except:
   trained_td3.save(TRAINED_MODEL_DIR + "/agent_td3_fundamentos") if if_using_td3 else None

## Treinando o algoritimo SAC

In [47]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": taxa_aprendizagem,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.01, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [48]:
start_time = time.time()
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=iteracao) if if_using_sac else None
end_time = time.time()

# Calcula o tempo decorrido em segundos
execution_time_seconds = end_time - start_time

# Calcula o tempo decorrido em minutos
execution_time_minutes = execution_time_seconds / 60

# Imprime o tempo de execução em minutos
print(f"Tempo de execução: {execution_time_minutes:.2f} minutos")

Tempo de execução: 0.00 minutos


In [49]:
try:
    trained_sac.save('/content/drive/Othercomputers/Meu laptop/Aprendizado_reforco_trading/Finrl/Finrl_fundamentalist_trader/trained_models/agent_sac_fundamentos') if if_using_sac else None

except:
    trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac_fundamentos") if if_using_sac else None